# Feature Engineering Testing Book

This is for experimenting with feature engineering processes

In [1]:
import numpy as np
import cudf as dd

In [ ]:
agg_func = ['mean', 'max', 'min', 'sum', 'std']

### Check Bureau merge and nulls

In [ ]:
bureau_balance = dd.read_parquet('raw_data/bureau_balance.parquet')
bureau = dd.read_parquet('raw_data/bureau.parquet')

In [ ]:

print("procecssing bureau balance")

## Build Average Bureau Balance
avg_bbalance = bureau_balance.select_dtypes('number') \
                    .groupby('SK_ID_BUREAU').agg(agg_func)

avg_bbalance.columns = ["_".join(x) for x in avg_bbalance.columns.ravel()]
avg_bbalance['MONTHS_BALANCE_std'] = avg_bbalance.MONTHS_BALANCE_std.fillna(0)

In [ ]:
avg_bbalance.columns

In [ ]:
bureau['DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_ENDDATE'].fillna(0)
bureau['DAYS_ENDDATE_FACT'] = bureau['DAYS_ENDDATE_FACT'].fillna(0)
bureau['AMT_CREDIT_MAX_OVERDUE'] = bureau['AMT_CREDIT_MAX_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM_LIMIT'] = bureau['AMT_CREDIT_SUM_LIMIT'].fillna(0)
bureau['AMT_ANNUITY'] = bureau['AMT_ANNUITY'].fillna(0)

avg_bureau = bureau.merge(avg_bbalance, how='left', 
                              left_on='SK_ID_BUREAU', 
                              right_index=True)

avg_bureau.set_index('SK_ID_CURR')

In [ ]:
avg_bureau.isnull().any()

I guess there are a lot of bureau IDs that don't have records in the bureau balance table

### Lets look at the behavioural data in cc_balance / payments and pc_balance

In [ ]:
# check the other historical records to see if we should stack and merge
cc_balance = dd.read_parquet('raw_data/cc_balance.parquet')

In [ ]:
cc_balance.isnull().any()

In [ ]:
# so we have time series data
cc_balance.SK_ID_PREV.unique == len(cc_balance)

In [ ]:
cc_balance['AMT_DRAWINGS_ATM_CURRENT'] = cc_balance['AMT_DRAWINGS_ATM_CURRENT'].fillna(0)
cc_balance['AMT_DRAWINGS_OTHER_CURRENT'] = cc_balance['AMT_DRAWINGS_OTHER_CURRENT'].fillna(0)
cc_balance['AMT_DRAWINGS_POS_CURRENT'] = cc_balance['AMT_DRAWINGS_POS_CURRENT'].fillna(0)
cc_balance['AMT_INST_MIN_REGULARITY'] = cc_balance['AMT_INST_MIN_REGULARITY'].fillna(0)
cc_balance['AMT_PAYMENT_CURRENT'] = cc_balance['AMT_PAYMENT_CURRENT'].fillna(0)
cc_balance['CNT_DRAWINGS_ATM_CURRENT'] = cc_balance['CNT_DRAWINGS_ATM_CURRENT'].fillna(0)
cc_balance['CNT_DRAWINGS_OTHER_CURRENT'] = cc_balance['CNT_DRAWINGS_OTHER_CURRENT'].fillna(0)
cc_balance['CNT_DRAWINGS_POS_CURRENT'] = cc_balance['CNT_DRAWINGS_POS_CURRENT'].fillna(0)
cc_balance['CNT_INSTALMENT_MATURE_CUM'] = cc_balance['CNT_INSTALMENT_MATURE_CUM'].fillna(0)

In [ ]:
sum_cc_balance = cc_balance.drop('SK_ID_PREV', axis=1) \
                        .select_dtypes('number').groupby('SK_ID_CURR') \
                        .agg(agg_func)

sum_cc_balance.columns = ["_".join(x) for x in sum_cc_balance.columns.ravel()]


In [ ]:
type(sum_cc_balance)

In [ ]:
sum_cc_balance.isnull().any().index.to_arrow().tolist()

### POS Balance

In [ ]:
pc_balance = dd.read_parquet('raw_data/pc_balance.parquet')

In [ ]:
pc_balance.dtypes

In [ ]:
pc_balance.isnull().any()

In [ ]:
pc_balance['CNT_INSTALMENT'] = pc_balance['CNT_INSTALMENT'].fillna(0)
pc_balance['CNT_INSTALMENT'] = pc_balance['CNT_INSTALMENT_FUTURE'].fillna(0)

In [ ]:
sum_pc_balance = pc_balance.drop('SK_ID_PREV', axis=1).select_dtypes('number').groupby('SK_ID_CURR') \
                .agg(agg_func)

sum_pc_balance.columns = ["_".join(x) for x in sum_pc_balance.columns.ravel()]

In [ ]:
sum_pc_balance.columns

In [ ]:
sum_pc_balance.isnull().any()

In [ ]:
sum_pc_balance['CNT_INSTALMENT_std'] = sum_pc_balance['CNT_INSTALMENT_std'].fillna(0)
sum_pc_balance['CNT_INSTALMENT_FUTURE_mean'] = sum_pc_balance['CNT_INSTALMENT_FUTURE_mean'].fillna(0)
sum_pc_balance['CNT_INSTALMENT_FUTURE_max'] = sum_pc_balance['CNT_INSTALMENT_FUTURE_max'].fillna(0)
sum_pc_balance['CNT_INSTALMENT_FUTURE_min'] = sum_pc_balance['CNT_INSTALMENT_FUTURE_min'].fillna(0)
sum_pc_balance['CNT_INSTALMENT_FUTURE_sum'] = sum_pc_balance['CNT_INSTALMENT_FUTURE_sum'].fillna(0)
sum_pc_balance['CNT_INSTALMENT_FUTURE_std'] = sum_pc_balance['CNT_INSTALMENT_FUTURE_std'].fillna(0)
sum_pc_balance['SK_DPD_std'] = sum_pc_balance['SK_DPD_std'].fillna(0)
sum_pc_balance['SK_DPD_DEF_std'] = sum_pc_balance['SK_DPD_DEF_std'].fillna(0)
sum_pc_balance['MONTHS_BALANCE_std'] = sum_pc_balance['MONTHS_BALANCE_std'].fillna(0)

In [ ]:
sum_pc_balance.isnull().any()

### Process Payments

In [ ]:
payments = dd.read_parquet('raw_data/payments.parquet')

In [ ]:
payments.isnull().any()

In [ ]:
payments['DAYS_ENTRY_PAYMENT'] = payments['DAYS_ENTRY_PAYMENT'].fillna(0)
payments['AMT_PAYMENT'] = payments['AMT_PAYMENT'].fillna(0)

In [ ]:
sum_payments = payments.drop('SK_ID_PREV', axis=1)
sum_payments['PAYMENT_PERC'] = sum_payments.AMT_PAYMENT / sum_payments.AMT_INSTALMENT
sum_payments['PAYMENT_PERC'] = sum_payments['PAYMENT_PERC'].fillna(0)
sum_payments['PAYMENT_DIFF'] = sum_payments.AMT_INSTALMENT - sum_payments.AMT_PAYMENT
sum_payments['DPD'] = sum_payments.DAYS_ENTRY_PAYMENT - sum_payments.DAYS_INSTALMENT
sum_payments['DBD'] = sum_payments.DAYS_INSTALMENT - sum_payments.DAYS_ENTRY_PAYMENT
    
# turn negatives into 0
sum_payments['DPD'] = sum_payments['DPD'].map(lambda x: x if x > 0 else 0)
sum_payments['DBD'] = sum_payments['DBD'].map(lambda x: x if x > 0 else 0)


In [ ]:
sum_payments.isnull().any()

In [ ]:
sum_payments = sum_payments.select_dtypes('number').groupby('SK_ID_CURR') \
                .agg(agg_func)
sum_payments.columns = ["_".join(x) for x in sum_payments.columns.ravel()]

In [ ]:
sum_payments.columns

In [ ]:
sum_payments['NUM_INSTALMENT_VERSION_std'] = sum_payments['NUM_INSTALMENT_VERSION_std'].fillna(0)
sum_payments['NUM_INSTALMENT_NUMBER_std'] = sum_payments['NUM_INSTALMENT_NUMBER_std'].fillna(0)
sum_payments['DAYS_INSTALMENT_std'] = sum_payments['DAYS_INSTALMENT_std'].fillna(0)
sum_payments['DAYS_ENTRY_PAYMENT_std'] = sum_payments['DAYS_ENTRY_PAYMENT_std'].fillna(0)
sum_payments['AMT_INSTALMENT_std'] = sum_payments['AMT_INSTALMENT_std'].fillna(0)
sum_payments['AMT_PAYMENT_std'] = sum_payments['AMT_PAYMENT_std'].fillna(0)
sum_payments['PAYMENT_PERC_std'] = sum_payments['PAYMENT_PERC_std'].fillna(0)
sum_payments['PAYMENT_DIFF_std'] = sum_payments['PAYMENT_DIFF_std'].fillna(0)
sum_payments['DPD_std'] = sum_payments['DPD_std'].fillna(0)
sum_payments['DBD_std'] = sum_payments['DBD_std'].fillna(0)

In [ ]:
# check overlaps with bureau_balance on the SK_ID_CURR
np.intersect1d(cc_balance.SK_ID_PREV.to_arrow().to_pylist(), pc_balance.SK_ID_PREV.to_arrow().to_pylist())

In [ ]:
np.intersect1d(cc_balance.SK_ID_PREV.to_arrow().to_pylist(), payments.SK_ID_PREV.to_arrow().to_pylist())

In [ ]:
cc_balance[cc_balance['SK_ID_PREV']==1000018]

In [ ]:
payments[payments['SK_ID_PREV']==1000018]

In [ ]:
np.intersect1d(pc_balance.SK_ID_PREV.to_arrow().to_pylist(), payments.SK_ID_PREV.to_arrow().to_pylist())

I think my hypothesis the behaviour data is for different types of transactions is wrong. Can revisit if there is time otherwise lets fillnas to 0 then continue to modelling for now

### Explore Prev

In [ ]:
import pandas as pd

In [ ]:
prev = pd.read_parquet('raw_data/prev.parquet')

In [ ]:
from feature_engineering import process_prev

In [ ]:
sum_prev = process_prev(prev, agg_func)

In [ ]:
sum_prev.isnull().any()[sum_prev.isnull().any()==True]

In [ ]:
len(sum_prev.isnull().any()[sum_prev.isnull().any()==True].index)

## Exploring training frame

In [ ]:
import pandas as pd
train = pd.read_parquet('raw_data/train.parquet')
test = pd.read_parquet('raw_data/test.parquet')

train_target = train['TARGET']
unified = pd.concat([train.drop('TARGET', axis=1), test])
unified.NAME_TYPE_SUITE

In [ ]:
train = dd.read_parquet('raw_data/train.parquet')
test = dd.read_parquet('raw_data/test.parquet')

train_target = train['TARGET']
unified = dd.concat([train.drop('TARGET', axis=1), test])

In [ ]:
unified.EMERGENCYSTATE_MODE.unique()

In [ ]:
from feature_engineering import process_unified

In [ ]:
unified = process_unified(unified)

In [ ]:
unified.isnull().any()[unified.isnull().any()==True].index

In [ ]:
np.median(unified['YEARS_BUILD_AVG'])

In [ ]:
unified.AMT_REQ_CREDIT_BUREAU_YEAR.unique()

## Exploring merged nulls

In [2]:
train_feats = dd.read_parquet('data_eng/feats/train_feats.parquet')

In [3]:
train_feats.isnull().any()[train_feats.isnull().any()==True].index.to_arrow().tolist()

[]